## Reading in the Data

In [2]:
import pipeline as pl
import imp

In [3]:
imp.reload(pl)
claims = pl.read_all()

In [5]:
claims.head()

,Unnamed: 0,reimb2010,reimb2008,reimb2009,age2010,male,race,heart.failure,kidney,cancer,...,OfficeVisit,EyeExam,EKG,xray,CTScan,PhysicalTherapy,Ambulance,acuity,costTrend,monthsWithClaims
0,0,390,320.0,360,97,0,White,0,0,0,...,4,1,0,1,0,0,0,0.235294,-0.018856,9
1,1,970,58800.0,2740,79,0,White,1,1,0,...,12,0,2,8,1,0,1,0.853591,-0.027265,21
2,2,5630,510.0,1580,87,0,White,1,0,0,...,10,0,1,2,1,1,0,0.175115,0.496742,20
3,3,3480,2930.0,49330,79,0,White,1,1,0,...,14,1,1,6,4,3,3,0.709147,0.214955,19
4,4,920,1500.0,1650,85,1,White,1,0,1,...,16,2,1,3,0,1,0,0.174603,0.231568,21
